# About This Assignment

Design and implement a complete **Natural Language Processing (NLP)** pipeline for
advanced sequence-to-sequence tasks using the Sherlock Holmes dataset, including:
-  text summarisation
- semantic search
- thematic analysis 

The focus is on understanding the process, implementing modular steps, and critically evaluating outcomes.

**Objective** 

To write a comprehensive report detailing the development, findings, and
results of your (NLP) pipeline, focusing on:
- How design choices influenced performance.
- Challenges encountered at each stage.
- Insights gained from the dataset and NLP methods used.
- Suggest improvements for each component of the pipeline.


# About this Data

- This collection features all the stories and novels of Sherlock Holmes by Arthur Conan Doyle. 
- Within the Sherlock folder, you'll find multiple .txt files, each containing a unique story.

# Importing neccesary libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing the dataset